# Ejercicio de la Práctica 5: Automatic Differentation (AD) in Pytorch


En esta ejercicio nos centraremos en la implementación de AD en Pytorch. En concreto, en el submódulo [TORCH.AUTOGRAD](https://pytorch.org/docs/stable/autograd.html) 

Si no tienes instalado Pytorch, lo primero que has de hacer es instalarlo. Una forma sencilla de hacerlo es en el **Powershell Prompt** de **Anaconda**, escribiendo:

**conda install pytorch torchvision torchaudio pytorch-cuda=11.6 -c pytorch -c nvidia**

##  AD en Pytorch

Carga Pytorch escribiendo 

**import torch**

In [14]:
# Completar aquí
import torch as pt
# --------------------


El objetivo de este ejercicio es hacer los mismos cálculos que hicimos en la práctica de AD en TensorFlow pero ahora en Pytorch

Veamos un primer ejemplo elemental: derivada de la función $y = x^2$ en $x=3$.

In [15]:
# Completar aquí
x = pt.tensor(3.0, requires_grad=True)
"""
El argumento 'requires_grad=True' indica que se quiere calcular la derivada de la función con respecto a esta variable.
"""

# Definir la función
y = x**2

# Calcular la derivada
y.backward()

# Obtener el valor de la derivada
print(f"Derivada de y respecto de x = {x.grad.item()}")
# --------------------


Derivada de y respecto de x = 6.0


Consideremos ahora una función de dos variables

$$
f(x_1, x_2) = x_1^2(x_1 + x_2)
$$

Vamos a calcular su gradiente en $x_1 = 2$, $x_2 = 3$. 

In [16]:
# Completar aquí
# Definir las variables
x_1 = pt.tensor(2.0, requires_grad=True)
x_2 = pt.tensor(3.0, requires_grad=True)

# Definir la función f(x1,x2)
f = x_1**2 * (x_1 + x_2)

# Calcular el gradiente
f.backward()

# Obtener el vector del gradiente en x_1 y x_2
gradiente_x_1 = x_1.grad.item()
gradiente_x_2 = x_2.grad.item()

# Convertir en un tupla para mejor reprensentación
gradiente_f = (gradiente_x_1, gradiente_x_2)

print(f"Gradiente de f(x1,x2) = {gradiente_f}")
# --------------------


Gradiente de f(x1,x2) = (24.0, 4.0)


Calculamos ahora su matriz Hessiana en el mismo punto y los valores propios de dicha matriz

In [17]:
from torch.autograd.functional import hessian

def f(x):
    return x[0]**2 * (x[0] + x[1])

x = torch.tensor([2.0, 3.0], requires_grad=True)

hess_f = hessian(f, x)
print("Hessian matrix:\n", hess_f)

from numpy.linalg import eig

eigenvalues, _ = eig(hess_f)

print(f"valores propios de la matriz hessiana de f en (2, 3) = \n {eigenvalues}")

Hessian matrix:
 tensor([[18.,  4.],
        [ 4.,  0.]])
valores propios de la matriz hessiana de f en (2, 3) = 
 [18.848858  -0.8488578]


Veamos ahora un modelo más elaborado. Consideremos la función vectorial 
$$
y = x * w  + b
$$
donde $x$ es un vector fila de $4$ componentes, $w$ es una matriz $4\times 3$ y $b$ un vector columna de $3$ componentes.

Define la función anterior en **pytorch** y asigna los siguientes valores a la variables:

1)  $x = [[1., 2., 3., 4.]]$
2)  $w$ valores aleatorios
3)  $b$ unos

In [18]:
# Completar aquí
x = pt.tensor([[1., 2., 3., 4.]], requires_grad=True)
w = pt.randn((4, 3), requires_grad=True)
b = pt.ones((3, ), requires_grad=True)

# Definir la función y = x * w + b
y = pt.matmul(x, w) + b # Multiplicación de matrices y suma del bias

print(f"x = {x}")
print(f"w = {w}")
print(f"b = {b}")
print(f"y = {y}")
# --------------------


x = tensor([[1., 2., 3., 4.]], requires_grad=True)
w = tensor([[ 0.3106, -0.5961, -2.3737],
        [-0.1981,  0.1905, -0.2271],
        [-0.7525,  0.1824,  0.3437],
        [-2.1758, -1.1129, -0.2312]], requires_grad=True)
b = tensor([1., 1., 1.], requires_grad=True)
y = tensor([[-10.0460,  -3.1197,  -1.7214]], grad_fn=<AddBackward0>)


Además de la función $y = x * w + b$ consideramos la función de pérdida

$$
\text{loss } = \frac{1}{3} \sum_{j=1}^3 (y_j - (y_{\text{label}})_j)^2
$$
donde $y_{\text{label}} = [[1., 2., 3.]]$

Define la función loss en pytorch

In [21]:
# Completar aquí
# Definir la etiqueta y_label
y_label = pt.tensor([[1., 2., 3.]], requires_grad=True)
print(f"y_label = {y_label}")

loss = pt.mean((y - y_label) ** 2)
print(f"loss = {loss}")
# --------------------


y_label = tensor([[1., 2., 3.]], requires_grad=True)
loss = 56.839298248291016


Calcula en pytorch los gradientes de la función loss, primero respecto a $y$ y después respecto a $x$

In [27]:
# Completar aquí
# Retener los gradientes de y
y.retain_grad()

# Recalcular la pérdida para hacer backward
loss = pt.mean((y - y_label) ** 2)

# Limpiar los gradientes de x e y
x.grad.zero_()
w.grad.zero_()
b.grad.zero_()

# Calcular el backward
loss.backward(retain_graph=True) # El parámetro permite calcular varios gradientes

# Obtener los gradientes respecto a 'y' y a 'x'
gradiente_y = y.grad.clone()
gradiente_x = x.grad.clone()

gradiente_loss = (gradiente_y, gradiente_x)
print(f"gradiente_loss = {gradiente_loss}")
# --------------------


gradiente_loss = (tensor([[-36.8201, -17.0657, -15.7380]]), tensor([[ 7.2183,  1.5234,  3.8368, 20.5487]]))
